# Data Wrangling & Cleaning

## Introduction

K2 engineering is an enterprise that makes consulting especially on environmental issues. At this moment K2  is interested in understanding and  predicting noise. Noise is an important variable that affects city planning, wild life, and human health. 

K2 has many stations which are used to measure noise and in some cases other environmental variables. So they have delivered us several csv files with information from the monitoring of environmental noise from various sectors of the city of Bogotá, which must be analyzed to meet the proposed objective: understand environmental noise and predict it.

<h4> In this file we can find:</h4>

* [Checking initial data](#Checking-initial-data): Initial evaluation of the data
* [Information Merge](#Information-merge): Integration of information in a single CSV file
* [Cleaning of special characters](#Cleaning): Elimination of special characters in the name of the station, latitude and longitude
* [Adding Columns](#Adding-Columns): Adding fields for temporal analysis(year, month, day, time, etc.) and spatial analysis. 
* [Summary Available data](#Summary-Available-data): Initial summary of available data: information area, number of records, minimum date, maximum date

In [4]:
import pandas as pd
import numpy as np
import glob 
import datetime

***

## Checking initial data

K2 Ingenieria gave us two sets of data, grouped under the names **SDA** and **aerocivil**, for both several csv files with the same structure are available:

- **Estación**:	Station name
- **Serial**:	Monitor equipment serial
- **Variable**:	Monitored variable
- **Unidad**:	Unit of measure
- **Componente**:	Measurement theme
- **Fecha**:	Measurement date and time
- **Valor**:	Measurement value
- **Ponderación**:	Weighting of the noise measurement
- **Tipo**:	Frequency type of noise measurement


In [2]:
# Indicating the path where the files are located
input_path='../data/raw/'
# Indicating path to save results
output_path = '../data/interim/'

**SDA information**

Checking SDA file structure - just an example file

In [3]:
# SDA information
directory_path = input_path+'/sda/*.csv'
files = glob.glob(directory_path,recursive = True)
print("Number of files SDA", len(files))

df_sda_test = pd.read_csv(files[1],delimiter =';',header = 0 , dtype={'Valor':float}, encoding='utf_8')
df_sda_test.describe(include = 'all')

Number of files SDA 369


,Estación,Serial,Variable,Unidad,Componente,Fecha,Valor,Ponderación,Tipo
count,93960,93960.0,93960,93960,93960,93960,93960.000000,93960,93960
unique,1,NaN,5,1,1,540,NaN,3,35
top,"﻿""CAI 20 de Julio""",NaN,L90,dBA,Ruido,2019-05-27 21:00:00,NaN,Lin,1/3 Oct 20kHz
freq,93960,NaN,18900,93960,93960,174,NaN,86400,2700
mean,NaN,11354.0,NaN,NaN,NaN,NaN,54.237512,NaN,NaN
std,NaN,0.0,NaN,NaN,NaN,NaN,18.188914,NaN,NaN
min,NaN,11354.0,NaN,NaN,NaN,NaN,3.100000,NaN,NaN
25%,NaN,11354.0,NaN,NaN,NaN,NaN,43.900000,NaN,NaN
50%,NaN,11354.0,NaN,NaN,NaN,NaN,54.800000,NaN,NaN
75%,NaN,11354.0,NaN,NaN,NaN,NaN,66.025000,NaN,NaN


In [4]:
# SDA Weighting of the noise measurement
df_sda_test.groupby(['Componente','Ponderación','Serial','Unidad']).agg({'Valor': ['mean', 'min', 'max','median']})

Valor                    
                                           mean   min    max median
Componente Ponderación Serial Unidad                               
Ruido      A           11354  dBA     66.698574  31.3  103.5   65.6
           C           11354  dBA     84.176111  52.8  122.0   81.3
           Lin         11354  dBA     52.710230   3.1  102.7   53.1

**Aerocivil information**

Checking Aerocivil file structure - just an example file

In [5]:
# Aerocivil information
directory_path = input_path+'/aerocivil/*.csv'
files = glob.glob(directory_path,recursive = True)
print("Number of files Aerocivil", len(files))
df_aerocivil_test = pd.read_csv(files[5],delimiter =';',header = 0 , dtype={'Valor':float}, encoding='utf_8')
df_aerocivil_test.describe(include ='O')

Number of files Aerocivil 6


,Estación,Serial,Variable,Unidad,Componente,Fecha,Ponderación,Tipo
count,522053,522053,522053,522053,522053,522053,474640,474640
unique,1,1,12,7,2,7919,3,40
top,"﻿""Estación Monitoreo Ruido Inteligente 7""",CE0117,Leq,dBA,Ruido,2020-07-11 20:37:17,Lin,Leq
freq,522053,522053,316400,474640,474640,66,284760,47470


In [6]:
df_aerocivil_test.groupby(['Componente','Ponderación','Serial','Unidad']).agg({'Valor': ['mean', 'min', 'max','median']})

Valor                           \
                                           mean          min         max   
Componente Ponderación Serial Unidad                                       
Ruido      A           CE0117 dBA    -64.642496 -9999.000000   88.599998   
           C           CE0117 dBA     71.376260 -9999.000000  108.699997   
           Lin         CE0117 dBA     41.400332     2.086823   80.127221   

                                                 
                                         median  
Componente Ponderación Serial Unidad             
Ruido      A           CE0117 dBA     52.099998  
           C           CE0117 dBA     71.800003  
           Lin         CE0117 dBA     44.358663

***

## Information-merge

To integrate the information of interest in a single file, we generate a function that takes the csv files from the directory one by one, they are saved in an auxiliary dataframe that is later added to a dataframe, saving all the information. Due to performance and computational restrictions, a new one is generated every 20 files.

In [15]:
# merge by groups of 20 files and generate one output
def join_csv_files(search_path,df_size=20):    
    files = glob.glob(search_path,recursive = True)
    source = search_path.split('/')[-2]
    print(source.upper())
    print('Number of files to process: ',len(files))
    df = pd.DataFrame()
    counter = 0
    file_counter = 0

    for filepath in files:
        #  Create auxiliary dataframe
        df_aux = pd.read_csv(filepath,sep =';',header = 0, encoding='utf_8') 
        
        #  Fields that do not generate interest for analysis
        if any(x in ['Serial','Unidad'] for x in df_aux.columns):
            df_aux.drop(['Serial','Unidad'],axis=1,inplace=True)
            
        # Information merge        
        df = df.append(df_aux)
        counter += 1
#  Generate interim files integrated by parts       
        if counter%df_size==0 or counter==len(files):
            df.to_csv(output_path+str(file_counter)+'_'+source+'_data.csv',index=False, header=True, encoding='utf_8',sep =';')
            df = pd.DataFrame()
            print('Dataframe saved '+str(file_counter))
            file_counter+=1
    print('Process finished successfully, check in output_path:',str(file_counter)+'_'+source+'_data.csv')

**Information merge SDA**

In [14]:
# Generate interim files integrated by parts - source:SDA
join_csv_files(input_path +'sda/*.csv')

SDA
Number of files to process:  369
20
Dataframe saved 0
40
Dataframe saved 1
60
Dataframe saved 2
80
Dataframe saved 3
100
Dataframe saved 4
120
Dataframe saved 5
140
Dataframe saved 6
160
Dataframe saved 7
180
Dataframe saved 8
200
Dataframe saved 9
220
Dataframe saved 10
240
Dataframe saved 11
260
Dataframe saved 12
280
Dataframe saved 13
300
Dataframe saved 14
320
Dataframe saved 15
340
Dataframe saved 16
360
Dataframe saved 17
369
Dataframe saved 18
Process finished successfully, check in output_path: 19_sda_data.csv


**Information merge Aerocivil**

In [16]:
%time
# Generate interim files integrated by parts - source: Aerocivil
join_csv_files(input_path +'aerocivil/*.csv')

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.2 µs
AEROCIVIL
Number of files to process:  6
Dataframe saved 0
Process finished successfully, check in output_path: 1_aerocivil_data.csv


**Integrated SDA + Aerocivil**

In [17]:
# Generate interim file integrated SDA + Aerocivil
join_csv_files(output_path +'*.csv')

INTERIM
Number of files to process:  20
Dataframe saved 0
Process finished successfully, check in output_path: 1_interim_data.csv


In [22]:
# load interim file integrated SDA + Aerocivil
source_data_df = pd.read_csv(output_path +'0_interim_data.csv', dtype =str,encoding='utf_8',sep=';')
print("Columns:", source_data_df.columns)
print("Total Rows:", source_data_df.shape[0])
source_data_df.head()

Columns: Index(['Estación', 'Variable', 'Componente', 'Fecha', 'Valor', 'Ponderación',
       'Tipo'],
      dtype='object')
Total Rows: 49279239


,Estación,Variable,Componente,Fecha,Valor,Ponderación,Tipo
0,"﻿""Estación Monitoreo Ruido Inteligente 13""",Dirección del Viento,Meteorología,2020-03-12 15:46:17,260.92817679558,NaN,NaN
1,"﻿""Estación Monitoreo Ruido Inteligente 13""",Leq,Ruido,2020-03-12 15:46:17,11.035490234953,Lin,1/3 Oct 3.15kHz
2,"﻿""Estación Monitoreo Ruido Inteligente 13""",Leq,Ruido,2020-03-12 15:46:17,11.035490234953,Lin,1/3 Oct 6.3kHz
3,"﻿""Estación Monitoreo Ruido Inteligente 13""",Leq,Ruido,2020-03-12 15:46:17,11.035490234953,Lin,1/3 Oct 6.3Hz
4,"﻿""Estación Monitoreo Ruido Inteligente 13""",Leq,Ruido,2020-03-12 15:46:17,11.035490234953,Lin,1/3 Oct 5kHz


**Spatial information from the monitoring station**

In [2]:
# Station Aerocivil
lat_long_aerocivil = pd.read_csv('../data/raw/ubicaciones/estacion_aerocivil.csv', header=0,sep=';', encoding='utf-8')
lat_long_aerocivil['Origen'] = 'aerocivil'
lat_long_aerocivil.head()

,nombre,nombre corto,latitud,longitud,altitud,altura suelo,Origen
0,Estación radar,ER1,"4,699","-74,105",NaN,NaN,aerocivil
1,Est 2.,Est2,"4,681339","-74,133675",NaN,NaN,aerocivil
2,Est. 1,Est1,"4,731777","-74,184502",NaN,NaN,aerocivil
3,Estación Monitoreo Ruido Inteligente 3,EMRI 3,"4,674752778","-74,12220556",2554.0,9.0,aerocivil
4,Estación Monitoreo Ruido Inteligente 1,EMRI 1,"4,686777778","-74,11775556",2626.0,4.0,aerocivil


In [18]:
# Station SDA
lat_long_sda = pd.read_csv('../data/raw/ubicaciones/estacion_sda.csv', header=0,sep=';', encoding='utf-8')
lat_long_sda['Origen'] = 'SDA'
lat_long_sda.rename(columns={'nombreCorto':'nombre corto'}, inplace=True)
lat_long_sda.head()

,nombre,nombre corto,latitud,longitud,altitud,altura suelo,Origen
0,Estación Meteorología,Est. Met.,"4,663","-74,062",NaN,NaN,SDA
1,Test2,TET2,"4,32","-74,25",2500.0,NaN,SDA
2,CAI Normandia,CAINORM,"4,6702139","-74,10749444",NaN,NaN,SDA
3,CAI Oneida,C.Oneida,"4,618888889","-74,145",NaN,NaN,SDA
4,CAI Quirigua,CAIQUI,"4,706286111","-74,10720278",NaN,NaN,SDA


**Merge station location information**

In [19]:
# Integrate station location information in df
lat_long_stations = lat_long_sda.copy().append(lat_long_aerocivil.copy())
lat_long_stations.head()

,nombre,nombre corto,latitud,longitud,altitud,altura suelo,Origen
0,Estación Meteorología,Est. Met.,"4,663","-74,062",NaN,NaN,SDA
1,Test2,TET2,"4,32","-74,25",2500.0,NaN,SDA
2,CAI Normandia,CAINORM,"4,6702139","-74,10749444",NaN,NaN,SDA
3,CAI Oneida,C.Oneida,"4,618888889","-74,145",NaN,NaN,SDA
4,CAI Quirigua,CAIQUI,"4,706286111","-74,10720278",NaN,NaN,SDA


***

## Cleaning

Remove special characters

**Name of monitoring stations**

In [23]:
# List of stations
source_data_df['Estación'].unique()

array(['\ufeff"Estación Monitoreo Ruido Inteligente 13"',
       '\ufeff"Estación Monitoreo Ruido Inteligente 7"',
       '\ufeff"CAI 20 de Julio"', '\ufeff"CAI 7 de Agosto"',
       '\ufeff"CAI Tejar"', '\ufeff"CAI Venecia"',
       '\ufeff"CAI Villa del Prado"', '\ufeff"CAI Villa Nidia"',
       '\ufeff"Cruz Roja"', '\ufeff"Edificio Marly"',
       '\ufeff"Edificio Profesional"', '\ufeffFontibón',
       '\ufeff"Hotel Morrison"', '\ufeffRestrepo',
       '\ufeff"Santa Cecilia"', '\ufeffSDA', '\ufeff"CAI Alamos"',
       '\ufeff"CAI Americas"', '\ufeff"CAI Aurora"', '\ufeff"CAI Claret"',
       '\ufeff"CAI Galerias"', '\ufeff"CAI Jaboque"',
       '\ufeff"CAI Las Ferias"', '\ufeff"CAI Navarra"',
       '\ufeff"CAI Normandia"', '\ufeff"CAI Quirigua"',
       '\ufeff"CAI Rincon"', '\ufeff"CAI Roma"',
       '\ufeff"CAI San Victorino"', '\ufeff"CAI Serena"'], dtype=object)

In [28]:
# # 1. Remove special character when loading the source file
source_data_df['Estación'] = source_data_df['Estación'].str.replace('"','')
# due to problems with enconding, some additional relacement needs to occur:
source_data_df['Estación'] = source_data_df['Estación'].str.replace(u'\ufeff','')

In [29]:
# List of stations
print(source_data_df['Estación'].unique())
print(len(source_data_df['Estación'].unique()))

['Estación Monitoreo Ruido Inteligente 13'
 'Estación Monitoreo Ruido Inteligente 7' 'CAI 20 de Julio'
 'CAI 7 de Agosto' 'CAI Tejar' 'CAI Venecia' 'CAI Villa del Prado'
 'CAI Villa Nidia' 'Cruz Roja' 'Edificio Marly' 'Edificio Profesional'
 'Fontibón' 'Hotel Morrison' 'Restrepo' 'Santa Cecilia' 'SDA' 'CAI Alamos'
 'CAI Americas' 'CAI Aurora' 'CAI Claret' 'CAI Galerias' 'CAI Jaboque'
 'CAI Las Ferias' 'CAI Navarra' 'CAI Normandia' 'CAI Quirigua'
 'CAI Rincon' 'CAI Roma' 'CAI San Victorino' 'CAI Serena']
30


In [30]:
source_data_df.to_csv('../data/cleaned_data/clean_sda_aerocivil.csv',index=False, header=True, encoding='utf_8',sep =';')

**latitude and longitude**

In [20]:
lat_long_stations.dtypes

nombre           object
nombre corto     object
latitud          object
longitud         object
altitud         float64
altura suelo    float64
Origen           object
dtype: object

The latitude and longitude data, useful for generating spatial analysis must be in float, the required decimal separator is not as needed:

In [21]:
# Replacing decimal separator in latitude and longitude to be able to convert to the corresponding data type
lat_long_stations['latitud']=lat_long_stations['latitud'].str.replace(',','.')
lat_long_stations['longitud']=lat_long_stations['longitud'].str.replace(',','.')

# Rename column name to make merge
lat_long_stations.rename(columns={'nombre' : 'Estación'},inplace =True)

In [22]:
lat_long_stations.to_csv("../data/cleaned_data/lat_long_stations.csv", sep =';', header=True,encoding='utf-8')

***

## Adding Columns

For temporal analysis we must do some transformations to our dataset and add columns with different time options.

In [52]:
source_data_df['Fecha'] = pd.to_datetime(source_data_df['Fecha'])

In [53]:
source_data_df["Year"] = source_data_df['Fecha'].dt.year

In [54]:
source_data_df['Month'] =  source_data_df['Fecha'].dt.to_period('M')

In [55]:
source_data_df["Week"] = source_data_df['Fecha'].dt.isocalendar().week

In [56]:
source_data_df['Hour'] = pd.to_datetime(source_data_df['Fecha']).dt.hour

In [57]:
source_data_df["Day"]= source_data_df['Fecha'].dt.day_name()

In [58]:
source_data_df['Date_no_hour'] =  source_data_df['Fecha'].dt.to_period('D')

In [ ]:
source_data_df['Minuto'] = pd.DatetimeIndex(source_data_df['Fecha']).minute

In [86]:
# Save source file with cleanup and transformations
source_data_df.to_csv("../data/cleaned_data/clean_sda_aerocivil_date_v0.csv",sep=';',header= True, encoding='utf-8',index=False)

***

## Summary Available data

Since we have in a single dataset the information of all the stations available for analysis, we can observe a rough summary of the information to be processed.

In [82]:
# Number of records and data period available per station
record_summary_df = source_data_df.groupby(['Estación','Componente' ]).agg({ 'Estación' : ['count'],'Date_no_hour' : [ 'min','max']})
record_summary_df.columns = record_summary_df.columns.map('_'.join)
record_summary_df = record_summary_df.reset_index()
record_summary_df['Nro dias'] =  record_summary_df['Date_no_hour_max'] - record_summary_df['Date_no_hour_min']

In [83]:
# Rename columns
record_summary_df.rename(columns={'Estación': 'Information zone',
                                    'Componente': 'Component',
                                    'Estación_count': 'Number of Records',
                                    'Date_no_hour_min': 'Minimum Date',
                                    'Date_no_hour_max': 'Maximum Date',
                                    'Nro dias': 'Information Period(Days)'},inplace =True)
record_summary_df.to_csv("../data/cleaned_data/record_summary_station.csv",sep=';',header= True, encoding='utf-8',index=False)
record_summary_df.head()

,Information zone,Component,Number of Records,Minimum Date,Maximum Date,Information Period(Days)
0,CAI 20 de Julio,Ruido,2949805,2019-04-10,2020-08-06,<484 * Days>
1,CAI 7 de Agosto,Ruido,306078,2019-11-08,2020-08-05,<271 * Days>
2,CAI Alamos,Ruido,2207711,2019-05-08,2020-08-05,<455 * Days>
3,CAI Americas,Ruido,2069654,2019-05-08,2020-08-06,<456 * Days>
4,CAI Aurora,Ruido,360101,2019-03-27,2019-08-12,<138 * Days>


In [7]:
record_summary_df= pd.read_csv("../data/cleaned_data/record_summary_station.csv",sep=';',header= 0, encoding='utf-8')
record_summary_df.sort_values(by='Number of Records', ascending = False)

,Information zone,Component,Number of Records,Minimum Date,Maximum Date,Information Period(Days)
17,CAI Venecia,Ruido,2990065,2019-04-10,2020-08-06,<484 * Days>
0,CAI 20 de Julio,Ruido,2949805,2019-04-10,2020-08-06,<484 * Days>
26,Estación Monitoreo Ruido Inteligente 7,Ruido,2561921,2020-04-16,2020-07-17,<92 * Days>
21,Edificio Marly,Ruido,2318626,2019-06-06,2020-08-06,<427 * Days>
18,CAI Villa Nidia,Ruido,2255629,2017-06-30,2020-08-05,<1132 * Days>
5,CAI Claret,Ruido,2212548,2019-05-08,2020-08-06,<456 * Days>
31,Santa Cecilia,Ruido,2210585,2019-05-08,2020-08-07,<457 * Days>
2,CAI Alamos,Ruido,2207711,2019-05-08,2020-08-05,<455 * Days>
10,CAI Normandia,Ruido,2176801,2019-05-08,2020-08-06,<456 * Days>
12,CAI Rincon,Ruido,2173467,2019-05-08,2020-08-06,<456 * Days>


In [8]:
print("number of information zones",record_summary_df.shape[0])

number of information zones 32


In [85]:
# Verify that all stations have latitude and longitude data
station_merge1 = record_summary_df.merge(lat_long_stations,how ='left',left_on='Information zone',right_on= 'nombre',indicator =True)
station_merge1[station_merge1['_merge']== 'left_only']

,Information zone,Component,Number of Records,Minimum Date,Maximum Date,Information Period(Days),nombre,nombre corto,latitud,longitud,altitud,altura suelo,Origen,_merge


In [13]:
record_summary_df.groupby(["Component"])[['Number of Records']].sum()

,Number of Records
Component,
Meteorología,355353
Ruido,48923886


In [24]:
print(round((355353/48923886)*100,2), "%")

0.73 %


In [41]:
import re
record_summary_df['days'] = record_summary_df['Information Period(Days)'].str.replace(r'\D+', '').astype('int')

In [42]:
record_summary_df['days'].describe()

count      32.000000
mean      363.093750
std       195.050094
min        35.000000
25%       285.500000
50%       419.500000
75%       456.000000
max      1132.000000
Name: days, dtype: float64

After the integration of the files, cleaning and transformation carried out, the following stand out:

- All stations have latitude and longitude information.
- Only 0.73% of the data correspond to meteorological data
- Most of the stations report more than a year of information, however, there are exceptions that must be carefully reviewed, such is the case of `CAI Villa Nidia`, `Estación Monitoreo Ruido Inteligente 13` and `Estación Monitoreo Ruido Inteligente 7`.